In [80]:
import pandas as pd
import os
import duckdb

In [ ]:
# fileOpen = pd.read_parquet('./data/2025_01.parquet')

# fileOpen['hora_arribo'] = fileOpen['hora_arribo'].astype(str).str.slice(0, 8)
# fileOpen['hora_retiro'] = fileOpen['hora_retiro'].astype(str).str.slice(0, 8)

# fileOpen.to_parquet('./data/2025_01.parquet')

In [68]:
with duckdb.connect("C:/Users/gerym/Documents/mcp/mcp_ecobici/bike_sharing2.db") as con:
    d = con.execute("""
    SELECT * 
    EXCLUDE(hora_retiro,hora_arribo), 
    fecha_retiro_completa::TIME as hora_retiro,
    fecha_arribo_completa::TIME as hora_arribo
    FROM bike_trip
    """).df()

    a = con.execute("""
        SELECT 
            fecha_arribo,
            DATE_TRUNC('month', fecha_arribo::DATE) AS first_day_of_month,
            day_of_the_week, 
            SUM(distance_meters)::INTEGER AS distance_meters,
            ROUND(SUM(time_between_trips), 2) AS time_between_trips,
            COUNT(*) AS total_trips 
        FROM d 
        GROUP BY 1, 2, 3 
        ORDER BY fecha_arribo ASC
    """).df()

    # d['fecha_arribo'] = d['fecha_arribo_completa'].dt.strftime('%Y-%m-%d')


In [79]:
a.to_dict('records')[0]

{'fecha_arribo': Timestamp('2025-01-01 00:00:00'),
 'first_day_of_month': Timestamp('2025-01-01 00:00:00'),
 'day_of_the_week': 'Wednesday',
 'distance_meters': 28819155,
 'time_between_trips': 298364.08,
 'total_trips': 16677}

In [75]:
from typing import List
from sqlmodel import Field, SQLModel
from typing import Optional
import datetime

class TripSummary(SQLModel, table=False):
    """
    Represents a summary of bike trips for reporting or analytics.
    Not a database table.
    """
    fecha_arribo: Optional[datetime.datetime] = None
    first_day_of_month: Optional[datetime.datetime] = None
    day_of_the_week: Optional[str] = None
    distance_meters: Optional[int] = None
    time_between_trips: Optional[float] = None
    total_trips: Optional[int] = None

def to_trip_summary_dicts(df) -> List[dict]:
    # Ensure correct types for each column
    df = df.copy()
    df['fecha_arribo'] = pd.to_datetime(df['fecha_arribo'])
    df['first_day_of_month'] = pd.to_datetime(df['first_day_of_month'])
    df['day_of_the_week'] = df['day_of_the_week'].astype(str)
    df['distance_meters'] = df['distance_meters'].astype(int)
    df['time_between_trips'] = df['time_between_trips'].astype(float)
    df['total_trips'] = df['total_trips'].astype(int)

    # Convert each row to TripSummary and then to dict
    return [
        TripSummary(
            fecha_arribo=row['fecha_arribo'],
            first_day_of_month=row['first_day_of_month'],
            day_of_the_week=row['day_of_the_week'],
            distance_meters=row['distance_meters'],
            time_between_trips=row['time_between_trips'],
            total_trips=row['total_trips']
        ).dict()
        for _, row in df.iterrows()
    ]

trip_summaries = to_trip_summary_dicts(a)
trip_summaries

C:\Users\gerym\AppData\Local\Temp\ipykernel_26916\157117113.py:37: DeprecationWarning: 
        🚨 `obj.dict()` was deprecated in SQLModel 0.0.14, you should
        instead use `obj.model_dump()`.
        
  ).dict()


[{'fecha_arribo': Timestamp('2025-01-01 00:00:00'),
  'first_day_of_month': Timestamp('2025-01-01 00:00:00'),
  'day_of_the_week': 'Wednesday',
  'distance_meters': 28819155,
  'time_between_trips': 298364.08,
  'total_trips': 16677},
 {'fecha_arribo': Timestamp('2025-01-13 00:00:00'),
  'first_day_of_month': Timestamp('2025-01-01 00:00:00'),
  'day_of_the_week': 'Monday',
  'distance_meters': 128236488,
  'time_between_trips': 968436.73,
  'total_trips': 67203},
 {'fecha_arribo': Timestamp('2025-01-14 00:00:00'),
  'first_day_of_month': Timestamp('2025-01-01 00:00:00'),
  'day_of_the_week': 'Tuesday',
  'distance_meters': 131853119,
  'time_between_trips': 983355.12,
  'total_trips': 67928},
 {'fecha_arribo': Timestamp('2025-01-15 00:00:00'),
  'first_day_of_month': Timestamp('2025-01-01 00:00:00'),
  'day_of_the_week': 'Wednesday',
  'distance_meters': 136467123,
  'time_between_trips': 1029273.13,
  'total_trips': 71048},
 {'fecha_arribo': Timestamp('2025-01-16 00:00:00'),
  'first_

In [12]:
# In your context, if you want to update the DataFrame 'one_bike' with a new 'fecha_arribo' column from 'fecha_arribo_completa':
with duckdb.connect("C:/Users/gerym/Documents/mcp/mcp_ecobici/bike_sharing2.db") as con:
    one_bike = duckdb.sql(""" 
    SELECT * EXCLUDE(hora_retiro,hora_arribo) FROM bike_trip
    """).df()

    one_bike['fecha_arribo'] = one_bike['fecha_arribo_completa'].dt.strftime('%Y-%m-%d')

    one_bike2 = con.execute("""
        SELECT fecha_arribo, SUM(distance_meters) distance_meters
        FROM one_bike
        GROUP BY fecha_arribo
        ORDER BY fecha_arribo ASC
    """).df()

one_bike2

CatalogException: Catalog Error: Table with name bike_trip does not exist!
Did you mean "duckdb_constraints"?

In [22]:
with duckdb.connect("C:/Users/gerym/Documents/mcp/mcp_ecobici/bike_sharing2.db") as con:
    bike_trips = con.execute("SELECT * FROM bike_trips").df()

bike_trips time_between_trips,day_of_the_week,distance_meters

,genero_usuario,edad_usuario,bici,ciclo_estacion_retiro,fecha_retiro,ciclo_estacionarribo,fecha_arribo,fecha_retiro_completa,fecha_arribo_completa,time_between_trips,day_of_the_week,is_weekend,lat_start,lon_start,lat_end,lon_end,distance_meters,hora_retiro,hora_arribo
0,M,26,5180930,568,2024-12-31,572,2025-01-01,2024-12-31 23:57:02,2025-01-01 00:00:03,3.016667,Wednesday,0,19.418013,-99.144598,19.414677,-99.136894,889.014215,23:57:02,00:00:03
1,F,54,3653953,283,2024-12-31,596,2025-01-01,2024-12-31 23:51:40,2025-01-01 00:00:41,9.016667,Wednesday,0,19.397923,-99.167248,19.395205,-99.155049,1314.687764,23:51:40,00:00:41
2,M,38,7511322,034,2024-12-31,064,2025-01-01,2024-12-31 23:48:36,2025-01-01 00:00:59,12.383333,Wednesday,0,19.424728,-99.163348,19.412967,-99.166925,1360.437222,23:48:36,00:00:59
3,M,41,3804572,258,2024-12-31,023,2025-01-01,2024-12-31 23:54:11,2025-01-01 00:01:08,6.950000,Wednesday,0,19.416530,-99.179044,19.425280,-99.171099,1280.930290,23:54:11,00:01:08


In [15]:
with duckdb.connect("C:/Users/gerym/Documents/mcp/mcp_ecobici/bike_sharing1.db") as con:
    bike_stations = con.execute("SELECT * EXCLUDE(index), ciclo_estacionarribo as ciclo_estacion_retiro FROM bike_stations").df()

bike_stations.dtypes

ciclo_estacionarribo     object
alcaldia                 object
colonia                  object
zip_code                  int64
ciclo_estacion_retiro    object
dtype: object